In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import scipy.optimize as opt

In [112]:
### linprog

In [110]:
cond = pd.DataFrame(np.array([[15,4, 14],[-150,-200, -300],[15,-25, 0]]), columns = ['P', 'Q', 'restr'], index = ['fat', 'calor', 'price'])
cond

,P,Q,restr
fat,15,4,14
calor,-150,-200,-300
price,15,-25,0


In [97]:
obj = cond.loc['price'].values[[0,1]]
obj

array([15, 25])

In [99]:
lhs_ineq = cond.loc[['fat', 'calor'], ['P', 'Q']].values
lhs_ineq

array([[  15,    4],
       [-150, -200]])

In [100]:
rhs_ineq = cond.loc[['fat', 'calor'], ['restr']].values
rhs_ineq

array([[  14],
       [-300]])

In [101]:
opt = linprog(c=obj, A_ub = lhs_ineq, b_ub = rhs_ineq)
opt

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 35.0
              x: [ 6.667e-01  1.000e+00]
            nit: 2
          lower:  residual: [ 6.667e-01  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf]
                 marginals: [ 0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [-3.125e-01 -1.313e-01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [102]:
opt.x

array([0.66666667, 1.        ])

In [105]:
print(f'покупать продукта Р: {opt.x[0]:.2f} ({opt.x[0]/opt.x.sum()*100:.2f}%) и продукта Q: {opt.x[1]:.2f} ({opt.x[1] / opt.x.sum()*100:.2f})%')

покупать продукта Р: 0.67 (40.00%) и продукта Q: 1.00 (60.00)%


In [106]:
check = (cond.loc[['fat', 'calor','price'], ['P', 'Q']].values).dot(opt.x)
check

array([  14., -300.,   35.])

In [107]:
cond.merge(pd.Series(check, index = ['fat', 'calor', 'price'], name = 'solution'), left_index=True, right_index=True)

,P,Q,restr,solution
fat,15,4,14,14.0
calor,-150,-200,-300,-300.0
price,15,25,0,35.0


In [113]:
# optimize ???

In [2]:
def min_price(x):
    p, q = x
    return p * 15 + q * 25

In [29]:
constr1 = ({'type': 'ineq', 'fun': lambda p: 14  - p[0]*15 - p[1]* 4})
constr2 = ({'type': 'ineq', 'fun': lambda p: -300  + p[0]*150 + p[1]* 200})
constr =  [constr1, constr2]
constr

[{'type': 'ineq', 'fun': <function __main__.<lambda>(p)>},
 {'type': 'ineq', 'fun': <function __main__.<lambda>(p)>}]

In [27]:
bond = ((0,10000), (0,100000))

In [11]:
x0 = np.array([0, 0])

In [33]:
sol_opt = opt.minimize(min_price, [0,0], bounds = bond, constraints = constr)

In [34]:
sol_opt

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 35.00000000010623
       x: [ 6.667e-01  1.000e+00]
     nit: 2
     jac: [ 1.500e+01  2.500e+01]
    nfev: 6
    njev: 2

In [37]:
sol_opt.x

array([0.66666667, 1.        ])

In [41]:
print(f'покупать продукта Р: {sol_opt.x[0]:.2f} ({sol_opt.x[0]/sol_opt.x.sum()*100:.2f}%) и продукта Q: {sol_opt.x[1]:.2f} ({sol_opt.x[1] / sol_opt.x.sum()*100:.2f})%')

покупать продукта Р: 0.67 (40.00%) и продукта Q: 1.00 (60.00)%
